In [ ]:
import mbuild as mb
import numpy as np
import matplotlib.pyplot as plt
import fresnel
from cme_utils.manip.utilities import rotation_matrix_from_to as cme_rot

from draw_scene import visualize
from diffractometer import Diffractometer, camera_to_rot

In [ ]:
## Used for creating files to get diffraction data

#pdbname = "fcc10"
#pdbname = "bcc10"
pdbname = "sc10"

dirname = "example_inputs"

box = mb.Box(np.array([10,10,10])/10) # mbuild scales pdbs... 

pdb = mb.load(f"{dirname}/{pdbname}.pdb")

#pdb.save(f"{dirname}/{pdbname}.gsd", box=box, overwrite=True)

In [ ]:
d = Diffractometer()
d.load(pdb.xyz, box.maxs)

scene = visualize(pdb)

# play around with different camera positions just make sure that
# up and position aren't be the same
scene.camera = fresnel.camera.orthographic(position=(1,1,1), look_at=(0,0,0), up=(0,0,1), height=1.5)

mat = camera_to_rot(scene.camera)
a = np.array(scene.camera.look_at)
b = np.array(scene.camera.position)

cmemat = cme_rot(b-a,np.array([0,0,1]))

print(mat, "\n", cmemat)

dp = d.diffract(mat)
#idbig = d.circle_cutout(dp)
#dp[np.unravel_index(idbig, (d.N, d.N))] = np.log10(d.bot)
plt.imshow(dp, cmap = "jet")
plt.axis("off")
plt.show()

#zooms = [i for i in range(1,d.N) if d.N % i == 0]
#print(zooms)
#for z in zooms:
#    d.zoom = z
#    dp = d.diffract(mat)
#    #idbig = d.circle_cutout(dp)
#    #dp[np.unravel_index(idbig, (d.N, d.N))] = np.log10(d.bot)
#    plt.imshow(dp, cmap = "jet")
#    plt.axis("off")
#    plt.show()

peaks = [i for i in range(6)]
for p in peaks:
    d.peak_width = p
    dp = d.diffract(mat)
    #idbig = d.circle_cutout(dp)
    #dp[np.unravel_index(idbig, (d.N, d.N))] = np.log10(d.bot)
    plt.imshow(dp, cmap = "jet")
    plt.axis("off")
    plt.show()


fresnel.preview(scene)

In [ ]:
help(cme_rot)

In [ ]:
rotmats[-1]

In [ ]:
from cme_utils.manip import utilities
n_v = 20
def prep_matrices():
    ga = np.pi * (3.0 - 5 ** 0.5)
    theta = ga * np.arange(n_v - 3)
    z = np.linspace(
        1 - 1.0 / (n_v - 3), 1.0 / (n_v - 1 - 3), n_v - 3
    )
    radius = np.sqrt(1.0 - z * z)
    points = np.zeros((n_v, 3))
    points[:-3, 0] = radius * np.cos(theta)
    points[:-3, 1] = radius * np.sin(theta)
    points[:-3, 2] = z
    points[-3] = np.array([0, 0, 1])
    points[-2] = np.array([0, 1, 1])
    points[-1] = np.array([1, 1, 1])
    return [utilities.rotation_matrix_from_to(i, np.array([0, 0, 1])) for i in points]

rotmats = prep_matrices()

In [ ]:
def vector_projection(u, v):
    """
    Projection of u onto v
    
    Parameters
    ----------
    u,v : numpy.ndarray (3,), vectors
    
    Returns
    -------
    numpy.ndarray (3,), projection of u onto v
    """
    return v * np.dot(u, v)/np.linalg.norm(v)

def unit_vector(vector):
    """
    Returns the unit vector of the vector.
    """
    return vector / np.linalg.norm(vector)

def get_angle(u, v):
    """
    Find angle between u and v
    
    Parameters
    ----------
    u,v : numpy.ndarray (3,), vectors
    
    Returns
    -------
    float, angle between u and v in radians
    """
    u = unit_vector(u)
    v = unit_vector(v)
    angle = np.arccos(np.clip(np.dot(u,v), -1.0, 1.0))
    if angle != angle:
        # Catches nan values
        return 0.0
    return angle

def camera_to_rot(camera):
    """
    Given a fresnel camera object, compute the rotation matrix
    
    Parameters
    ----------
    camera : fresnel.camera, camera in fresnel scene
    
    Returns
    -------
    numpy.ndarray (3,3), rotation matrix
    """
    pos = scene.camera.position
    look_at = scene.camera.look_at

    cam_vec = np.array(pos)-np.array(look_at)

    # axis vectors
    xvec = np.array([1,0,0])
    yvec = np.array([0,1,0])
    zvec = np.array([0,0,1])

    # Project the camera vector into the xy, yz, and xz planes
    # by subtracting the projection of the plane normal vector
    cam_xy = cam_vec - vector_projection(cam_vec, zvec)
    cam_yz = cam_vec - vector_projection(cam_vec, xvec)
    cam_xz = cam_vec - vector_projection(cam_vec, yvec)

    # find the angles betwen the camera vector projections and the axes vectors
    # alpha is in the yz, beta xz, gamma xy
    alpha = get_angle(cam_yz, yvec)
    beta = get_angle(cam_xz, zvec)
    gamma = get_angle(cam_xy, xvec)
    
    return rot_mat(alpha, beta, gamma)

def rot_mat(alpha, beta, gamma):
    """
    Given angles alpha, beta, and gamma, compute the rotation matrix
    
    Parameters
    ----------
    alpha, beta, gamma : float, angles about the x, y, and z axes in radians
    
    Returns
    -------
    numpy.ndarray (3,3), rotation matrix
    """
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(alpha), -np.sin(alpha)],
        [0, np.sin(alpha), np.cos(alpha)]
    ])
    Ry = np.array([
        [np.cos(beta), 0, np.sin(beta)],
        [0, 1, 0],
        [-np.sin(beta), 0, np.cos(beta)]
    ])
    Rz = np.array([
        [np.cos(gamma), -np.sin(gamma), 0],
        [np.sin(gamma), np.cos(gamma), 0],
        [0, 0, 1]
    ])
    return np.dot(np.dot(Rx,Ry),Rz)

In [ ]:
# d.load sets the following

#print(d.box)
#for i in range(d.orig.shape[0]):
#    print(d.orig[i,:], d.image[i,:])

In [ ]:
#for i,mat in enumerate(rotmats):
#    print(i)
#    print(mat)

In [ ]:
#from scipy import interpolate, ndimage
#
#rot = rotmats[0]
#print(rot.shape)
#N = d.N / d.zoom
#inv_shear = d.calc_proj(rot)
#print(inv_shear.shape)
#xy = np.copy(np.dot(d.orig, rot)[:, 0:2])
#print(xy.shape)
#xy = np.dot(xy, inv_shear.T)
#print(xy.shape)
#xy = d.pbc_2d(xy, N)
#print(xy.shape)
#im = d.bin(xy, N)
#print(im.shape)
#
#dp = np.fft.fft2(im)
#print(dp.shape)
#dp = ndimage.fourier.fourier_gaussian(dp, d.peak_width / d.zoom)
#dp = np.fft.fftshift(dp)
#dp = np.absolute(dp)
#dp *= dp
#dp = d.scale(dp)
#print(dp.shape)
#dp = d.shear_back(dp, inv_shear)
#print(dp.shape)
#dp /= dp.max()
#dp[dp < d.bot] = d.bot